In [55]:
!pip install razdel
!pip3 install -r "/content/drive/MyDrive/Colab Notebooks/dialogue-rusimplification/requirements.txt"
!pip install rnnmorph

!pip install git+https://github.com/huggingface/transformers

!git clone https://github.com/feralvam/easse.git
%cd /content/easse/
!pip install .


     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 194kB 14.3MB/s 
     |████████████████████████████████| 245kB 53.5MB/s 
     |████████████████████████████████| 112kB 51.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=875d82c6256ac0767b2f4eb50707b15e7a3bcd1afc7898d36dfa3b468687345f
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-jl63leo1
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-jl63leo1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.5.0.dev0-cp37-none-any.whl size=1972568 sha256=091e9ba912b36233f17aeb41ea127a3b6bd6fb13f327888ac348f20c38d0fbfc
  Store

## Loading data

In [9]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook
from razdel import tokenize

from rnnmorph.predictor import RNNMorphPredictor
from sklearn.model_selection import train_test_split
from easse.sari import corpus_sari
import json
PATH = '/content/drive/MyDrive/Colab Notebooks/dialogue-rusimplification'

In [79]:
# Read the data
df = pd.read_csv(os.path.join(PATH,'dev_sents.csv'))
pd.set_option('display.max_colwidth', 150)
pd.set_option('display.width', 500)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3406 entries, 0 to 3405
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     3406 non-null   int64 
 1   INPUT:source   3406 non-null   object
 2   OUTPUT:output  3406 non-null   object
dtypes: int64(1), object(2)
memory usage: 80.0+ KB


In [77]:
df.head()

,Unnamed: 0,INPUT:source,OUTPUT:output
0,3,"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички переименован в Советский.
1,4,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"В 1960 году вышла модель 172А. Отличие в хвостовом оперении, в руле направления и в креплении шасси"
2,5,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,В выпущенной в 1960 году модель имела изменения в хвостовом оперении и руле направления
3,6,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,Изменения: в хвосте и руле направления с обратной стреловидностью и крепления для поплавкового шасси.\r\n
4,7,1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"Модель 172А с другим хвостовым оперением, креплениями для поплавкового шасси рулём направления с обратной стреловидностью появилась в 1960 году."


## Text preprocess

Группируем данные по INPUT:





In [13]:
df_grouped = df.groupby('INPUT:source' ).agg({'OUTPUT:output': '(SEP)'.join})
input_corpus = df_grouped.index.values
output_corpus = [output_text.split('(SEP)') for output_text in df_grouped['OUTPUT:output'].values]


In [14]:
df_grouped.head(5)

,OUTPUT:output
INPUT:source,
1960 году была выпущена модель 172A. Изменения: хвостовое оперение и руль направления с обратной стреловидностью и крепления для поплавкового шасси.,"В 1960 году вышла модель 172А. Отличие в хвостовом оперении, в руле направления и в креплении шасси(SEP)В выпущенной в 1960 году модель имела изме..."
"14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.",14 декабря 1944 года рабочий посёлок Ички переименован в Советский.
"26 августа 2014 года Болдок принял участие в памятном матче Кубка Футбольной лиги против «Манчестер Юнайтед», в котором «донс» сенсационно обыграл команду Луи ван Гала со счётом 4:0.",26 августа 2014 года Болдок выиграл футбольный матч Кубка лиги у команды Луи ван Гала со счётом 4:0(SEP)26 августа 2014 года Болдок принял участие...
"Armory — клиент с различными функциями для повышения безопасности, является надстройкой, работающей поверх bitcoind, однако использует собственный формат хранения секретного ключа.","Armory - программа с различными функциями для повышения безопасности. Является дополнением, которое работает вместе с bitcoind, но использует свой..."
"Concert for Diana (рус. Концерт для Дианы) — концерт памяти, организованный в честь Дианы, принцессы Уэльской.","Concert for Diana - это концерт, посвящённый памяти принцессы Дианы Уэльской.(SEP)В честь Дианы, принцессы Уэльской ,организован концерт памяти Ди..."


In [15]:
#@lru_cache(maxsize=32)
def text_preprocess(corpus):
    predictor = RNNMorphPredictor(language="ru")
    new_corpus = []
    for sentence in tqdm_notebook(corpus, desc= 'preprocessing...'):
        sentence = ' '.join([predictor.predict([token.text])[0].normal_form 
                             if token.text.isnumeric()==False else token.text 
                             for token in list(tokenize(sentence))])
        new_corpus.append(sentence)
    return new_corpus


In [16]:
def dataframe_lemmatized(input_corpus, output_corpus):
    df_lemmatized = pd.DataFrame()
    input_text_lemmatized = text_preprocess(input_corpus)
    # returns list of lemmatized group of text which corresponds to one input text
    output_text_lemmatized = [text_preprocess(group) for group in output_corpus]

    df_lemmatized['input'] = input_text_lemmatized 
    df_lemmatized = pd.concat([df_lemmatized , pd.DataFrame(data = output_text_lemmatized, columns = ['output_1' , 
                                                                                                      'output_2' , 
                                                                                                      'output_3' , 
                                                                                                      'output_4' , 
                                                                                                      'output_5'])], axis=1)
    return df_lemmatized

Загружаем ранее лемматизированный текст:

In [31]:
try:
    df_lemmatized = pd.read_csv(os.path.join(PATH,'lemmatized_text.csv'))
    df_lemmatized = df_lemmatized.fillna(0)  
except FileNotFoundError:
    df_lemmatized = dataframe_lemmatized(input_corpus, output_corpus)
    df_lemmatized.to_csv(os.path.join(PATH, 'lemmatized_text.csv'))


In [32]:
df_lemmatized.head()

,Unnamed: 0,input,output_1,output_2,output_3,output_4,output_5
0,0,1960 год быть выпущенный модель 172 a . изменение : хвостовой оперение и руль направление с обратный стреловидность и крепление для поплавковый ша...,"в 1960 год выйти модель 172 а . отличие в хвостовой оперение , в руль направление и в крепление шасси",в выпущенный в 1960 год модель иметь изменение в хвостовой оперение и руль направление,изменение : в хвост и руль направление с обратный стреловидность и крепление для поплавковый шасси .,"модель 172 а с другой хвостовой оперение , крепление для поплавковый шасси руль направление с обратный стреловидность появиться в 1960 год .",0
1,1,"14 декабрь 1944 год рабочий посёлок ичка быть переименованный в рабочий посёлок советский , посол что поселковый совет стать называться советский .",14 декабрь 1944 год рабочий посёлок ичка переименованный в советский .,0,0,0,0
2,2,"26 август 2014 год болдок принять участие в памятный матч кубок футбольный лига против « манчестер юнайтед » , в который « донс » сенсационно обыг...",26 август 2014 год болдок выиграть футбольный матч кубок лига у команда луи ван гала со счёт 4 : 0,"26 август 2014 год болдок принять участие в футбольный матч против « манчестер юнайтед » , где победить со счёт 4 : 0 .",0,0,0
3,3,"armory — клиент с различный функция для повышение безопасность , являться надстройка , работающий поверх bitcoind , однако использовать собственны...","armory - программа с различный функция для повышение безопасность . являться дополнение , который работать вместе с bitcoind , но использовать сво...",0,0,0,0
4,4,"concert for diana ( рус . концерт для диана ) — концерт память , организованный в честь диана , принцесса уэльский .","concert for diana - это концерт , посвящённый память принцесса диана уэльский .","в честь диана , принцесса уэльский , организованный концерт память диана .","концерт для дианы- концентр память диана , принцесса уэльский .",0,0


In [33]:
df_lemmatized.shape

(1000, 7)

## Matching

Готовим данные для разметки и использования в обучении:

In [34]:
input = df_grouped.index.values
output = output_corpus
input_lemma = df_lemmatized['input'].values
output_lemma = df_lemmatized[['output_1' , 'output_2' , 'output_3' , 'output_4' , 'output_5']].values


Разбиваем датасет на тренировочную, валидационную и тестовую выборки

In [35]:
ids = np.arange(1000)
train_id, valid_id = train_test_split(ids, train_size = 0.7, random_state = 42)
valid_id, test_id = train_test_split(valid_id, train_size = 0.5,  random_state = 42)


train_input, train_input_lemma, train_output_lemma = input[train_id], input_lemma[train_id] , output_lemma[train_id]
valid_input, valid_input_lemma, valid_output_lemma = input[valid_id], input_lemma[valid_id] , output_lemma[valid_id]
test_input, test_input_lemma, test_output_lemma = input[test_id], input_lemma[test_id] , output_lemma[test_id]

print('Train data size:', train_input.shape , train_input_lemma.shape , train_output_lemma.shape )
print('Valid data size:', valid_input.shape , valid_input_lemma.shape , valid_output_lemma.shape )
print('Test data size:', test_input.shape , test_input_lemma.shape , test_output_lemma.shape )


Train data size: (700,) (700,) (700, 5)
Valid data size: (150,) (150,) (150, 5)
Test data size: (150,) (150,) (150, 5)


Записываем токены и тэги в JSON-файл в BIO формате:

In [43]:
#{"tokens": ["APPLICATION", "and", "Affidavit", "for", "Search", "Warrant", "as", "to", "The", "Matter", "of", "the", "Search", "of", "9", "Granite", "Street", ",", "#", "5", "(", "Attachments", ":", "#", "1", "Affidavit", "of", "James", "Keczkemethy)(Belpedio", ",", "Lisa", ")", "(", "Entered", ":", "12/15/2020", ")"], "tags": ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "I-MISC", "L-MISC", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]}

def matching(input, input_lemma, output_lemma, train_or_test):    
    with open(os.path.join(PATH,f'{train_or_test}.json') , 'w') as file:        
        for i, sentence in enumerate(input_lemma):
            # словарь для записи токенов и тэгов в json файл
            token_tag_dict = {}
            # список для записи тэгов
            tags = [] 
            
            # сложный лемматизированный текст
            input_lemma_sentence = input_lemma[i].split(' ')  

            # сложный исходный текст в виде списка токенов
            input_sentence = [token.text for token in list(tokenize(input[i]))]   

            # упрощенные предложения, в списке через запятую
            output_lemma_sentence = ' '.join(output_lemma[i][output_lemma[i]!=0])
            
            # все варианты упрощенного предложения в виде списка токенов
            output_lemma_sentence = [token.text for token in list(tokenize(output_lemma_sentence))]
            
            # записываем исходный текст в json    
            token_tag_dict['tokens'] = input_sentence
          
            for token in input_lemma_sentence: 
            # проверяем, если токен из исходного текста есть в упрощенном тексте,
                if token in output_lemma_sentence:
                  #  то ставим тэг I-YES
                    tags.append('I-YES')                              
                else:
                  # если нет токена в упрощенном тексте ставим O  
                    tags.append('O')                
            # записываем тэги в json
            token_tag_dict['tags'] = tags         
            # записываем все в файл
            json.dump(token_tag_dict , file, ensure_ascii = False)
            file.write('\n')
    

In [44]:
# train 70% of data
matching(train_input, train_input_lemma, train_output_lemma, 'train')

# valid 15% of data
matching(valid_input, valid_input_lemma, valid_output_lemma, 'valid')

# test 15% of data
matching(test_input, test_input_lemma, test_output_lemma, 'test')


## Training

In [57]:
%cd /content/drive/MyDrive/Colab Notebooks/dialogue-rusimplification
!wget "https://github.com/huggingface/transformers/blob/master/examples/token-classification/run_ner.py"

/content/drive/MyDrive/Colab Notebooks/dialogue-rusimplification
--2021-03-21 15:22:13--  https://github.com/huggingface/transformers/blob/master/examples/token-classification/run_ner.py
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘run_ner.py.1’

run_ner.py.1            [ <=>                ] 261.40K  --.-KB/s    in 0.03s   

2021-03-21 15:22:14 (8.04 MB/s) - ‘run_ner.py.1’ saved [267678]



In [46]:
!export MAX_LENGTH=128
!export BERT_MODEL=bert-base-multilingual-cased
!export OUTPUT_DIR=germeval-model
!export BATCH_SIZE=32
!export NUM_EPOCHS=3
!export SAVE_STEPS=750
!export SEED=1


In [58]:
!python ./run_ner.py \
  --model_name_or_path bert-base-multilingual-cased \
  --pad_to_max_length=True \
  --train_file train.json \
  --validation_file valid.json \
  --test_file test.json \
  --output_dir OUTPUT_DIR \
  --do_train \
  --do_eval \
  --do_predict

2021-03-21 15:22:37.662774: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
03/21/2021 15:22:38 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
03/21/2021 15:22:38 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=OUTPUT_DIR, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Mar21_15-22-38_87f64ee9eca0, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, 

## Predictions

In [61]:
def tag_to_text(path_to_file): 
    simplified = []
    with open(path_to_file, 'rb') as file:
        for i,row in enumerate(file):
            # проходим по файлу и записываем токены в в список.
            tags = np.array(str(row)[2:-3].split(' '))
            # в переменную записываем id токенов с тэгом I-YES
            I_YES_id, = np.where(tags=='I-YES')
            
            # список токенов в исходном тексте в тестовом наборе
            test_tokens_list = np.array([token.text for token in list(tokenize(test_input[i]))])
            
            # упрощеное предложение в виде списка токенов 
            test_simple = test_tokens_list[I_YES_id]
            # упрощенное предложение в виде строки
            test_simple = ' '.join(test_simple.tolist())

            # предложение записываем в список
            simplified.append(test_simple)
    return simplified

In [62]:
simplified = tag_to_text('./OUTPUT_DIR/test_predictions.txt')

## Evaluation

Данные для оценкки модели

In [63]:
# исходные предложения в виде списка
input_sents = test_input.tolist()
# упрощенные предложения в виде списка списков.
output_sents = np.array(output_corpus)[test_id].tolist()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Оцениваем модель по метрике SARI

In [69]:
from easse.sari import corpus_sari

corpus_sari(orig_sents=input_sents,  
            sys_sents=simplified, 
            refs_sents= output_sents)

24.009557417342943

Для визуальной оценки результатов запишем данные в датафрейм:

In [70]:
df_result = pd.DataFrame({'input':input_sents , 'output':output_sents ,'simplified': simplified})

In [80]:
df_result.head(5)

,input,output,simplified
0,Одним из главных событий в творческой биографии Высоцкого стала работа над ролью Гамлета в Театре на Таганке.,"[Роль Гамлета является одной из главных в биографии Высоцкого., Важнейшая роль Высоцкого - Гамлет в Театре на Таганке., Главное событие в биографи...",в биографии Высоцкого работа над ролью Гамлета в Театре на Таганке .
1,С первого удара Линдланд нанес Емельяненко рассечение над правым глазом и вошёл в клинч в попытке перевести бой в партер.,"[Лингдланд дал Емельяненко в глаз, а затем применил запрещённый приём., Линдланд попытался перевести бой в партер с Емельяненко., Линдланд в бою п...",С первого удара Линдланд нанес Емельяненко рассечение над правым глазом и вошёл в клинч в перевести бой в партер .
2,"Чтобы иметь возможность показать картину возможно большей зрительской аудитории, создатели фильма значительно смягчили на экране любовный контекст...","[В фильме меньше любовных сцен, чем в книге. Это сделано для того, чтобы показать его большому количеству зрителей., Создатели фильма смягчили люб...","показать картину большей , создатели фильма смягчили на любовный книги ."
3,"В своём докладе Немцов отмечает непрозрачность процедур при строительстве олимпийских объектов, отсутствие общественного контроля, закрытость инфо...","[В докладе Немцова отмечено о грубых нарушениях и скрытости точной информации при строительстве олимпийских объектов, В своём докладе Немцов отмеч...","В своём докладе Немцов непрозрачность процедур при строительстве олимпийских объектов , отсутствие общественного контроля , закрытость информации ..."
4,"Описанные Джимом Корбеттом, известным охотником на тигров, подобные случаи показывают, насколько драматичными и кровопролитными могут быть столкно...",[Охотник на тигров Корбетт описывал подобные случае столкновения тигра с людьми как драматичные и кровопролитные],"Описанные Джимом Корбеттом , охотником на тигров , случаи показывают , и кровопролитными могут быть столкновения тигра с людьми ."
